In [33]:
#import libraries
import pandas as pd
import numpy as np
import math
import random
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup    
import re
import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import time

In [34]:
df = pd.read_csv("/Users/suvojitdutta/Documents/PYTHON/Files/SentimentAnalysis/MovieReview.tsv", sep='\t', header=0, quoting=3, encoding = "ISO-8859-1", error_bad_lines=False)

In [35]:
print (stopwords.words("english") )

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [36]:
df.shape

(156060, 4)

In [37]:
df.dtypes

PhraseId       int64
SentenceId     int64
Phrase        object
Sentiment      int64
dtype: object

In [38]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters    
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [39]:
print ("Start time : " + time.strftime("%c"))
df['clean_review'] = df.Phrase.apply(review_to_words)
print ("End time : " + time.strftime("%c"))

Start time : Sat Nov  5 21:55:04 2016


/Users/suvojitdutta/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/Users/suvojitdutta/anaconda/lib/python3.5/site-packages/bs4/__init__.py:198: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Users/suvojitdutta/anaconda/lib/python3.5/site-packages/bs4/__init__.py:198: UserWarning: "b'movies'" looks like a filename, not markup. 

End time : Sat Nov  5 21:57:23 2016


In [40]:
df1 = df[['clean_review']]

In [41]:
clean_reviews =[]
clean_reviews = df.clean_review.values

In [42]:
print (clean_reviews)

[ 'series escapades demonstrating adage good goose also good gander occasionally amuses none amounts much story'
 'series escapades demonstrating adage good goose' 'series' ...,
 'avuncular chortles' 'avuncular' 'chortles']


In [43]:
print ("Creating the bag of words...\n")

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 10) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [44]:
train_data_features.shape

(156060, 10)

In [56]:
#Convert train_data_features in to a dataframe
df_train_input = pd.DataFrame(train_data_features, columns=vocab)

In [57]:
df_train_output = df[['Sentiment']]

In [58]:
#Merge Sentiment to the train dataframe
df_train = df_train_input.merge(right=df_train_output, how='inner', left_index=True, right_index=True, sort=False)
df_train

,characters,film,good,like,lrb,movie,one,rrb,story,time,Sentiment
0,0,0,2,0,0,0,0,0,1,0,1
1,0,0,1,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,2
5,0,0,1,0,0,0,0,0,0,0,2
6,0,0,0,0,0,0,0,0,0,0,2
7,0,0,1,0,0,0,0,0,0,0,2
8,0,0,0,0,0,0,0,0,0,0,2
9,0,0,1,0,0,0,0,0,0,0,2


In [45]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print (vocab)

['characters', 'film', 'good', 'like', 'lrb', 'movie', 'one', 'rrb', 'story', 'time']


In [46]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print (count, tag)

1882 characters
6733 film
2261 good
3190 like
2098 lrb
6241 movie
3784 one
2438 rrb
2539 story
1919 time


In [47]:
print (train_data_features)

[[0 0 2 ..., 0 1 0]
 [0 0 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [48]:
train_data_features.shape

(156060, 10)

In [49]:
clean_reviews.shape

(156060,)

In [28]:
print ("Training Random Forest starts at...." + time.strftime("%c"))

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, df["Sentiment"] )
print ("Training completed at...." + time.strftime("%c"))

Training Random Forest at....Fri Nov  4 23:33:08 2016


KeyboardInterrupt: 

In [29]:
#Process the test dataset

In [30]:
df_test = pd.read_csv("/Users/suvojitdutta/Documents/PYTHON/Files/SentimentAnalysis/TestReview.tsv", sep='\t', header=0, quoting=3, encoding = "ISO-8859-1", error_bad_lines=False)

In [31]:
df_test.shape

(66292, 3)

In [32]:
df_test.dtypes

PhraseId       int64
SentenceId     int64
Phrase        object
dtype: object

In [ ]:
print ("Start time : " + time.strftime("%c"))
df_test['clean_review'] = df_test.Phrase.apply(review_to_words)
print ("End time : " + time.strftime("%c"))

In [ ]:
clean_reviews_test =[]
clean_reviews_test = df_test.clean_reviews_test.values

In [ ]:
print ("Creating the bag of words for test dataset...\n")

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
test_data_features = vectorizer.fit_transform(clean_reviews_test)

# Numpy arrays are easy to work with, so convert the result to an 
# array
test_data_features = test_data_features.toarray()

In [ ]:
test_data_features.shape

In [ ]:
# Take a look at the words in the vocabulary from test dataset
vocab_test = vectorizer.get_feature_names()
print (vocab_test)

In [ ]:
# Sum up the counts of each vocabulary word from test dataset
dist = np.sum(test_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab_test, dist):
    print (count, tag)

In [ ]:
print (test_data_features)

In [ ]:
# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

In [ ]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

In [ ]:
# Use pandas to write the comma-separated output file
output.to_csv( "Model_test_result.csv", index=False, quoting=3 )